This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
#Check mongo is running
import pymongo
client = pymongo.MongoClient()
db = client.metrics

## Note - To Get this working:

* Install CUDA and associated libraries, setup path
* Install bleeding edge theano (from src)
* Make sure the THEANO_FLAGS are set correctly via the environment var, or via the ~/.theanorc file
* Install and compile bleeding edge Keras (from src)
* `export KERAS_BACKEND=theano`
* `export KERAS_IMAGE_DIM_ORDERING='th'`
* `sh <project_root>/shell_scipts/setup_environment.sh` to install additional dependencies
* **DO NOT SET UNROLL=True** when creating RNN's - causes max recursion issue

## Trouble-Shooting

* You may need to clean the theano cache. To do so thoroughly, run this command from the shell:
 * `theano-cache purge`

In [3]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

import keras
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging

import datetime

Using Theano backend.


## Load and Pre-Process Essays

In [4]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + "SkinCancer/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM_Stacked/"
cv_folder = root_folder + "CV_Data_Pickled/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
WARNING - No db name specified - should be either 'metrics_causal' or 'metrics'. Defaulting to 'metrics' 


In [5]:
cv_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/CV_Data_Pickled/'

In [6]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

870

In [7]:
test_config = get_config(test_folder)
tagged_essays_test = load_process_essays(**test_config)

218 files found
218 essays processed


In [8]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2019-10-09 00:27:24.445766


In [9]:
from numpy.random import shuffle
shuffle(tagged_essays)

## Prepare Tags

In [10]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

for essay in tagged_essays_test:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) 
                and not "Anaphor" in t 
                and not "other" in t 
                and not "rhetorical" in t))

vtags = set(regular_tags)
vtags.add(EMPTY_TAG)

cr_vtags = set(cr_tags)
cr_vtags.add(EMPTY_TAG)

len(unique_words)

1623

In [11]:
sorted(vtags)

['1', '11', '12', '2', '3', '4', '5', '50', '6', 'Empty']

In [12]:
len(cr_vtags)

50

In [13]:
sorted(cr_vtags)

['Causer:1->Result:2',
 'Causer:1->Result:3',
 'Causer:1->Result:4',
 'Causer:1->Result:5',
 'Causer:1->Result:50',
 'Causer:11->Result:12',
 'Causer:11->Result:3',
 'Causer:11->Result:4',
 'Causer:11->Result:5',
 'Causer:11->Result:50',
 'Causer:12->Result:12',
 'Causer:12->Result:2',
 'Causer:12->Result:3',
 'Causer:12->Result:4',
 'Causer:12->Result:5',
 'Causer:12->Result:50',
 'Causer:2->Result:1',
 'Causer:2->Result:11',
 'Causer:2->Result:2',
 'Causer:2->Result:3',
 'Causer:2->Result:4',
 'Causer:2->Result:5',
 'Causer:2->Result:50',
 'Causer:2->Result:6',
 'Causer:3->Result:11',
 'Causer:3->Result:2',
 'Causer:3->Result:4',
 'Causer:3->Result:5',
 'Causer:3->Result:50',
 'Causer:3->Result:6',
 'Causer:4->Result:11',
 'Causer:4->Result:12',
 'Causer:4->Result:4',
 'Causer:4->Result:5',
 'Causer:4->Result:50',
 'Causer:4->Result:6',
 'Causer:5->Result:12',
 'Causer:5->Result:4',
 'Causer:5->Result:5',
 'Causer:5->Result:50',
 'Causer:5->Result:6',
 'Causer:50->Result:2',
 'Causer

# Transform Essays into Training Data (Word Ids)

* Computes `xs`, `ys`, `ys_bytag` and `seq_lens`
* `ys_bytag` includes **all tags** and does **not** focus only on the most common tag
* `ys` only includes the most common tag (so we can use cross entropy)
* `seq_lens` is without the start and end tags included (so we have to map back and forth to maintain mappings)
* `ys_bytag` also excludes the START and END tokens

## Get Max Sequence Length, Generate All Ids

In [14]:
ix2tag = {}
for ix, t in enumerate(vtags):
    ix2tag[ix] = t

ix2crtag = {}
for ix, t in enumerate(cr_vtags):
    ix2crtag[ix] = t
    
generator = idGen(seed=1) # important as we zero pad sequences

maxlen = 0
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)
        
for essay in tagged_essays_test:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)

def ids2tags(ids):
    return [generator.get_key(j) for j in ids]

def lbls2tags(ixs):
    return [ix2tag[ix] for ix in ixs]
        
maxlen

119

In [15]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    ys_bytag_concept_sent = defaultdict(list)
    ys_bytag_cr_sent = defaultdict(list)
    seq_lens = []
    
    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set() # get all unique tags in sentence
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)  
                
                # Make sure to include Causer:<num> and Result:<num> tags, as we do for the parser model
                missing_tags = [t.replace("Causer:","").replace("Result:","") 
                                    for t in tags
                                ]
                # Filter to just concept codes that were missing
                missing_tags = set([t for t in missing_tags if t[0].isdigit() and "->" not in t])
                
                new_tags = tags.union(missing_tags)
                #if missing_tags:
                #    diff = missing_tags - tags
                #    if diff:
                #        print(diff, tags)

                tags = new_tags                
                
                unique_tags.update(tags)
                
                # remove unwanted tags, filter to concept tags
                concept_tags = vtags.intersection(tags)

                # encode ys with most common tag only
                if len(concept_tags) > 1:
                    most_common = max(concept_tags, key=lambda t: tag_freq[t])
                    concept_tags = set([most_common])
                if len(concept_tags) == 0:
                    concept_tags.add(EMPTY_TAG)

                one_hot = []
                for t in vtags:
                    if t in concept_tags:
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                y_seq.append(one_hot)
                #end for each word
            
            # sentence level tags
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_concept_sent[tag].append(1)
                else:
                    ys_bytag_concept_sent[tag].append(0)
            
            for tag in cr_vtags:
                if tag in unique_tags:
                    ys_bytag_cr_sent[tag].append(1)
                else:
                    ys_bytag_cr_sent[tag].append(0)
                
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = sequence.pad_sequences(xs, maxlen=maxlen)
    ys = sequence.pad_sequences(ys, maxlen=maxlen)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag_concept_sent, ys_bytag_cr_sent, seq_lens

## Create Train - Test Split

In [16]:
#Helper Functions
def collapse_results(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = pred_ys[1:-1]
        for pred_tag in pred_ys:
            pred_ys_by_tag[pred_tag].append(1)
            # for all other tags, a 0
            for tag in(vtags - set([EMPTY_TAG, pred_tag])):
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

In [17]:
#Helper Functions
def collapse_results_sentence_level(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = set(pred_ys[1:-1])
        for tag in vtags:
            if tag == EMPTY_TAG:
                continue
            if tag in pred_ys:
                pred_ys_by_tag[tag].append(1)
            else:
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

In [18]:
def train_dev_split(lst, dev_split, randomize=True):
    # random shuffle
    if randomize:
        shuffle(lst)
    num_training = int((1.0 - dev_split) * len(lst))
    return lst[:num_training], lst[num_training:]

In [19]:
%%time
# use this name for a different function later
from CrossValidation import cross_validation as cv

folds = cv(tagged_essays, CV_FOLDS)
fold2training_data = {}
fold2dev_data = {}
fold2test_data = {}

for i, (essays_TD, essays_VD) in enumerate(folds):
    # further split into train and dev test
    essays_train, essays_dev = train_dev_split(essays_TD, DEV_SPLIT)
    fold2training_data[i] = get_training_data(essays_train)
    fold2dev_data[i]      = get_training_data(essays_dev)
    # Test Data
    fold2test_data[i]     = get_training_data(essays_VD)

CPU times: user 6.6 s, sys: 404 ms, total: 7 s
Wall time: 6.99 s


In [20]:
with open(cv_folder + "td.dill", "wb") as f:
    dill.dump(fold2training_data, f)

with open(cv_folder + "devd.dill", "wb") as f:
    dill.dump(fold2dev_data, f)

with open(cv_folder + "vd.dill", "wb") as f:
    dill.dump(fold2test_data, f)

In [21]:
# Make sure the generator is incremented on the test data too
_,_,_,_,_, = get_training_data(tagged_essays_test)

## Load Glove 100 Dim Embeddings

In [22]:
# see /Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/WordVectors/pickle_glove_embedding.py
# for creating pre-filtered embeddings file
import pickle, os
from numpy.linalg import norm

embeddings_file = "/Users/simon.hughes/data/word_embeddings/glove.6B/sc_dict_glove.6B.100d.txt"
# read data file
with open(embeddings_file, "rb+") as f:
    cb_emb_index = pickle.load(f)

In [23]:
missed = set()
for wd in unique_words:
    if wd not in cb_emb_index:
        missed.add(wd)
print(len(missed), len(unique_words), 100.0 * round(len(missed)/  len(unique_words),4), "%")

16 1623 0.9900000000000001 %


### Construct Embedding Matrix

In [24]:
EMBEDDING_DIM = list(cb_emb_index.values())[0].shape[0]

def get_embedding_matrix(words, idgenerator, max_features, init='uniform', unit_length=False):
    embedding_dim = list(cb_emb_index.values())[0].shape[0]
    # initialize with a uniform distribution
    if init == 'uniform':
        # NOTE: the max norms for these is quite low relative to the embeddings
        embedding_matrix = np.random.uniform(low=-0.05, high=0.05,size=(max_features, embedding_dim))
    elif init =='zeros':
        embedding_matrix = np.zeros(shape=(max_features, embedding_dim), dtype=np.float32)
    elif init == 'normal':
        embedding_matrix = np.random.normal(mean, sd, size=(max_features, embedding_dim))
    else:
        raise Exception("Unknown init type")
    for word in words:
        i = idgenerator.get_id(word)
        embedding_vector = cb_emb_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    if unit_length:
        norms = np.linalg.norm(embedding_matrix, axis=1,keepdims=True)
        # remove 0 norms to prevent divide by zero
        norms[norms == 0.0] = 1.0
        embedding_matrix = embedding_matrix / norms
    return embedding_matrix

In [25]:
def score_predictions(model, xs, ys_by_tag, seq_len):
    preds = model.predict_classes(xs, batch_size=batch_size, verbose=0)   
    pred_ys_by_tag = collapse_results_sentence_level(seq_len, preds)
    class2metrics = ResultsProcessor.compute_metrics(ys_by_tag, pred_ys_by_tag)
    micro_metrics = micro_rpfa(class2metrics.values())
    return micro_metrics, pred_ys_by_tag

In [26]:
def pivot_predictions_to_dict(preds):
    pred_ys_by_tag = defaultdict(list)
    for i in range(preds.shape[0]):
        row = preds[i]
        for tag_ix, pred in enumerate(row):
            tag = ix2crtag[tag_ix]
            pred_ys_by_tag[tag].append(pred)
    return pred_ys_by_tag

In [27]:
def score_predictions_sent_level(model, xs, ys_by_tag, seq_len):
    preds = model.predict(xs, batch_size=batch_size, verbose=0)
    preds = np.where(preds >= 0.5, 1, 0)
    pred_ys_by_tag = pivot_predictions_to_dict(preds)
    class2metrics = ResultsProcessor.compute_metrics(ys_by_tag, pred_ys_by_tag)
    micro_metrics = micro_rpfa(class2metrics.values())
    return micro_metrics, pred_ys_by_tag

In [28]:
from keras.layers import Bidirectional
from datetime import datetime

def get_ts():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

def get_file_ts():
    return datetime.now().strftime('%Y%m%d_%H%M%S_%f')

embedding_size = EMBEDDING_DIM
hidden_size    = 128
out_size = len(vtags)
batch_size = 128

get_ts(), get_file_ts()

('2019-10-09 00:27:38.299743', '20191009_002738_299769')

## Train Bi-Directional LSTM With Glove Embeddings

In [29]:
max_features=len(generator.get_ids())+2 #Need plus one maybe due to masking of sequences
max_features

1627

In [30]:
def get_file_name(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    lcls = locals()
    s = ""
    for k, val in sorted(lcls.items(), key = lambda tpl: (0,tpl[0]) if tpl[0] == 'fold_ix' else (1,tpl[0])):
        s += "{key}-{val}_".format(key=k, val=str(val))
    return models_folder + s[:-1] + ".h5"

get_file_name(0, True, True, 2, 'sum', 128)

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Models/Bi-LSTM_Stacked/fold_ix-0_bi_directional-True_hidden_size-128_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.h5'

In [31]:
# merge_mode is Bi-Directional only
def evaluate_fold(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):

    if use_pretrained_embedding:
        embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
        embedding_layer = Embedding(max_features,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=True,
                                mask_zero=True) # If false, initialize unfound words with all 0's
    else:
        embedding_layer = Embedding(max_features, embedding_size, input_length=maxlen, trainable=True, mask_zero=True)

    if bi_directional:
        rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
    else:
        rnn_layer_fact = lambda : GRU(hidden_size, return_sequences=True, consume_less="cpu")
        
    model = Sequential()
    model.add(embedding_layer)
    for i in range(num_rnns):
        model.add(rnn_layer_fact())

    model.add(TimeDistributedDense(out_size))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")
    
    X_train, y_train, train_ys_bytag_con_sent, train_ys_by_tag_cr_sent, seq_len_train = fold2training_data[fold_ix]
    X_dev,   y_dev,   dev_ys_bytag_con_sent,   dev_ys_by_tag_cr_sent,   seq_len_dev   = fold2dev_data[fold_ix]
    X_test,  y_test,  test_ys_bytag_con_sent,  test_ys_by_tag_cr_sent,  seq_len_test  = fold2test_data[fold_ix]

    # init loop vars
    f1_scores = [-1]
    num_since_best_score = 0
    patience = 3
    best_weights = None

    for i in range(30):
    #for i in range(2):
        #print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
        epochs = 1 # epochs per training instance
        results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.0, verbose=0)
        micro_metrics,_ = score_predictions(model, X_dev, dev_ys_bytag_con_sent, seq_len_dev)

        #print(micro_metrics)
        #print()

        f1_score = micro_metrics.f1_score
        best_f1_score = max(f1_scores)
        if f1_score <= best_f1_score:
            num_since_best_score += 1
        else: # score improved
            num_since_best_score = 0
            best_weights = model.get_weights()

        f1_scores.append(f1_score)
        if num_since_best_score >= patience:
            #print("Too long since an improvement, stopping")
            break
    
    print("Fold[{ix}] - Best F1 Score={f1}".format(ix=fold_ix, f1=best_f1_score))
    
    # load best weights
    model.set_weights(best_weights)
    return model

## Hyper Param Tuning

In [32]:
def cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    fold2model = {}
    for i in range(CV_FOLDS):
        model = evaluate_fold(i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
        fname = get_file_name(i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
        model.save(fname)    
        fold2model[i] = model
    return fold2model

In [33]:
# temp
import warnings
warnings.filterwarnings("ignore")

i = 0
for use_pretrained_embedding in [True]:
    for bi_directional in [False]:
        for num_rnns in [1]:
            for merge_mode in ["sum"]:
                for hidden_size in [64]:

                    i += 1
                    print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
                          .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))
                    fold2model = cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
                    #print("MicroF1={micro_f1}".format(micro_f1=micro_f1))
                    print(get_ts())

[1] Params 2019-10-09 00:27:38.507334 - Embeddings=True, Bi-Direct=False Num_Rnns=1 Hidden_Size=64
Fold[0] - Best F1 Score=0.8818126148193508
Fold[1] - Best F1 Score=0.886653252850436
Fold[2] - Best F1 Score=0.8868778280542987
Fold[3] - Best F1 Score=0.8618581907090465
Fold[4] - Best F1 Score=0.8647781934794228
2019-10-09 00:46:13.121800


In [34]:
import warnings
warnings.filterwarnings("ignore")

i = 0
for use_pretrained_embedding in [True]:
    for bi_directional in [True]:
        for num_rnns in [2]:
            for merge_mode in ["sum"]:
                for hidden_size in [256]:

                    i += 1
                    print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
                          .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))
                    fold2model = cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
                    #print("MicroF1={micro_f1}".format(micro_f1=micro_f1))
                    print(get_ts())

[1] Params 2019-10-09 00:46:13.137016 - Embeddings=True, Bi-Direct=True Num_Rnns=2 Hidden_Size=256
Fold[0] - Best F1 Score=0.944686140459913
Fold[1] - Best F1 Score=0.9286199864038069
Fold[2] - Best F1 Score=0.9307958477508651
Fold[3] - Best F1 Score=0.9090909090909091
Fold[4] - Best F1 Score=0.9166666666666667
2019-10-09 04:07:22.708468


In [35]:
def load_model(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    fname = get_file_name(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
    return keras.models.load_model(fname)

def load_models(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    models_by_fold = {}
    for i in range(CV_FOLDS):
        model = load_model(i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
        models_by_fold[i] = model
    return models_by_fold

In [36]:
predicts_by_fold = {}
for fold_ix in range(CV_FOLDS):
    X_test,  y_test,  test_ys_bytag_con_sent,  test_ys_by_tag_cr_sent,  seq_len_test  = fold2test_data[fold_ix]
    model = fold2model[fold_ix]
    probs = model.predict_proba(X_test)
    predicts_by_fold[fold_ix] = probs

1670/1670 [==============================] - 12s    


## Get Test Data Predictions

In [37]:
# merge_mode is Bi-Directional only
def evaluate_test(num_rnns, merge_mode, hidden_size):
    embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
    embedding_layer = Embedding(max_features,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True,
                            mask_zero=True) # If false, initialize unfound words with all 0's
    rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
        
    model = Sequential()
    model.add(embedding_layer)
    for i in range(num_rnns):
        model.add(rnn_layer_fact())

    model.add(TimeDistributedDense(out_size))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")
    
    essays_train, essays_dev = train_dev_split(tagged_essays, DEV_SPLIT)
    X_train, y_train,  train_ys_bytag_con_sent,  train_ys_by_tag_cr_sent,  seq_len_train = get_training_data(essays_train)
    X_dev,   y_dev,    dev_ys_bytag_con_sent,    dev_ys_by_tag_cr_sent,    seq_len_dev   = get_training_data(essays_dev)

    # init loop vars
    f1_scores = [-1]
    num_since_best_score = 0
    patience = 3
    best_weights = None

    for i in range(30):
        print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
        results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, validation_split=0.0, verbose=0)
        micro_metrics,_ = score_predictions(model, X_dev, dev_ys_bytag_con_sent, seq_len_dev)

        print(micro_metrics)
        print()

        f1_score = micro_metrics.f1_score
        best_f1_score = max(f1_scores)
        if f1_score <= best_f1_score:
            num_since_best_score += 1
        else: # score improved
            num_since_best_score = 0
            best_weights = model.get_weights()

        f1_scores.append(f1_score)
        if num_since_best_score >= patience:
            #print("Too long since an improvement, stopping")
            break
    
    print("Fold[{ix}] - Best F1 Score={f1}".format(ix=fold_ix, f1=best_f1_score))
    
    # load best weights
    model.set_weights(best_weights)
    return model

In [38]:
test_model = evaluate_test(2, "sum", 256)

2019-10-09 04:08:29.296254: Epoch=0
Recall: 0.8498, Precision: 0.9088, F1: 0.8783, Accuracy: 0.9672, Codes:  1032

2019-10-09 04:13:28.344629: Epoch=1
Recall: 0.8953, Precision: 0.9185, F1: 0.9068, Accuracy: 0.9744, Codes:  1032

2019-10-09 04:18:01.067972: Epoch=2
Recall: 0.9322, Precision: 0.9188, F1: 0.9254, Accuracy: 0.9791, Codes:  1032

2019-10-09 04:22:34.180647: Epoch=3
Recall: 0.9331, Precision: 0.9260, F1: 0.9295, Accuracy: 0.9803, Codes:  1032

2019-10-09 04:27:07.028032: Epoch=4
Recall: 0.9118, Precision: 0.9401, F1: 0.9257, Accuracy: 0.9796, Codes:  1032

2019-10-09 04:31:39.930049: Epoch=5
Recall: 0.9360, Precision: 0.9288, F1: 0.9324, Accuracy: 0.9811, Codes:  1032

2019-10-09 04:36:12.990878: Epoch=6
Recall: 0.9360, Precision: 0.9351, F1: 0.9356, Accuracy: 0.9821, Codes:  1032

2019-10-09 04:40:46.184855: Epoch=7
Recall: 0.8934, Precision: 0.9370, F1: 0.9147, Accuracy: 0.9768, Codes:  1032

2019-10-09 04:45:19.557704: Epoch=8
Recall: 0.9409, Precision: 0.9195, F1: 0.930

In [39]:
X_test,  y_test,   test_ys_bytag_con_sent,   test_ys_by_tag_cr_sent,   seq_len_test = get_training_data(tagged_essays_test)
test_probs = test_model.predict_proba(X_test)

2097/2097 [==============================] - 15s    


In [40]:
test_probs.shape, len(test_ys_by_tag_cr_sent['Causer:3->Result:50'])

((2097, 119, 10), 2097)

# Stacked Model

## Generate Train and Test Data For Each Fold

In [41]:
def get_stacked_feat_from_probs(probs, max_feats, min_feats, average_feats, binary_feats, combo_feats):
    xs = []
    for i in range(len(probs)):
        preds = probs[i,:]
        max_preds = np.max(preds, axis=0)
        min_preds = np.max(preds, axis=0)
        mean_preds = np.mean(preds, axis=0)
        
        predicted_ixs = set(np.argwhere(max_preds > 0.5).flatten())
        binary = [0] * len(max_preds)
        for ix in predicted_ixs:
            binary[ix] = 1
        
        x = []
        if binary_feats:
            x += binary
        if max_feats:
            x += max_preds.tolist()
        if min_feats:            
            x += min_preds.tolist()
        if average_feats:
            x += mean_preds.tolist()

        # combination tags
        if combo_feats:
            ixs = ix2tag.keys()
            for a in ixs:
                for b in ixs:
                    if b < a:
                        if a in predicted_ixs and b in predicted_ixs:
                            x.append(1)
                        else:
                            x.append(0)
        xs.append(x)
    return np.asarray(xs)

def get_stacked_feats_by_fold(fold_ix, predicts_by_fold, max_feats, min_feats, average_feats, binary_feats, combo_feats):
    probs = predicts_by_fold[fold_ix]
    xs = get_stacked_feat_from_probs(probs, max_feats, min_feats, average_feats, binary_feats, combo_feats)
    return xs

### Loop Thru Each Fold, Merge the Xs and Ys from the Other Folds as TD, and then Use Fold as VD

In [42]:
from wordtagginghelper import merge_dictionaries

def generate_stacked_features(max_feats, min_feats, average_feats, binary_feats, combo_feats):
    stacked_feats_by_code = {}
    for fold_ix in range(CV_FOLDS):
        stacked_feats_by_code[fold_ix] = get_stacked_feats_by_fold(fold_ix, predicts_by_fold, max_feats, min_feats, average_feats, binary_feats, combo_feats)
    
    td_xs_by_fold = {}
    vd_xs_by_fold = {}

    td_ys_by_fold = {}
    vd_ys_by_fold = {}
    for vd_ix in range(CV_FOLDS):
        td = []

        td_ys = defaultdict(list)
        vd_ys = defaultdict(list)
        for td_ix in range(CV_FOLDS):
            if td_ix == vd_ix:
                continue
            xs = stacked_feats_by_code[td_ix]
            td.append(xs)

            _, _, _, td_ys_by_tag_cr_sent, _ = fold2test_data[td_ix]
            merge_dictionaries(td_ys_by_tag_cr_sent, td_ys)

        vd_xs_by_fold[vd_ix] = stacked_feats_by_code[vd_ix]
        td_xs_by_fold[vd_ix] = np.vstack(td)

        del td_ys[EMPTY_TAG]
        td_ys_by_fold[vd_ix] = td_ys

        _, _, _, vd_ys_by_tag_cr_sent, _ = fold2test_data[vd_ix]
        # make a copy (so can delete EMPTY tag)
        merge_dictionaries(vd_ys_by_tag_cr_sent, vd_ys)
        del vd_ys[EMPTY_TAG]    
        vd_ys_by_fold[vd_ix] = vd_ys
    return td_xs_by_fold, td_ys_by_fold, vd_xs_by_fold, vd_ys_by_fold

# Train Stacked Classifier

In [43]:
from wordtagginghelper import train_classifier_per_code, test_classifier_per_code
from sklearn.linear_model import LogisticRegression

def train_stacked_classifier(dual, penalty, C, max_feats, min_feats, average_feats, binary_feats, combo_feats):
    # capture param values
    fn_args  = dict(locals())
    
    td_xs_by_fold, td_ys_by_fold, vd_xs_by_fold, vd_ys_by_fold = generate_stacked_features(max_feats, min_feats, average_feats, binary_feats, combo_feats)
    
    fn_create_sent_cls  = lambda : LogisticRegression(dual=dual, C=C, penalty=penalty)

    cv_wd_td_ys_by_tag, cv_wd_td_predictions_by_tag = defaultdict(list), defaultdict(list)
    cv_wd_vd_ys_by_tag, cv_wd_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

    cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

    num_feats = []
    for i in range(CV_FOLDS):
        sent_td_xs = td_xs_by_fold[i]
        sent_vd_xs = vd_xs_by_fold[i]
            
        num_feats.append(sent_vd_xs.shape[1])
        
        sent_td_ys_bycode = td_ys_by_fold[i]
        sent_vd_ys_bycode = vd_ys_by_fold[i]

        tags = sent_td_ys_bycode.keys()

        tag2sent_classifier = train_classifier_per_code(sent_td_xs, sent_td_ys_bycode , fn_create_sent_cls, tags, verbose=False)
        td_sent_predictions_by_code \
            = test_classifier_per_code(sent_td_xs, tag2sent_classifier, tags )

        vd_sent_predictions_by_code \
            = test_classifier_per_code(sent_vd_xs, tag2sent_classifier, tags )

        merge_dictionaries(sent_td_ys_bycode, cv_sent_td_ys_by_tag)
        merge_dictionaries(sent_vd_ys_bycode, cv_sent_vd_ys_by_tag)
        merge_dictionaries(td_sent_predictions_by_code, cv_sent_td_predictions_by_tag)
        merge_dictionaries(vd_sent_predictions_by_code, cv_sent_vd_predictions_by_tag)

    sent_algo = str(fn_create_sent_cls())

    SUFFIX = ""
    CB_SENT_TD, CB_SENT_VD = "CR_SC_STACKED_TD" + SUFFIX, "CR_SC_STACKED_VD" + SUFFIX
    parameters = dict(config)
    parameters["extractors"] = []
    parameters["num_feats_MEAN"] = np.mean(num_feats)
    # merge in function args
    parameters.update(fn_args)

    sent_td_objectid = processor.persist_results(CB_SENT_TD, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, parameters, sent_algo)
    sent_vd_objectid = processor.persist_results(CB_SENT_VD, cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag, parameters, sent_algo)
    
    avg_f1 = float(processor.get_metric(CB_SENT_VD, sent_vd_objectid, __MICRO_F1__)["f1_score"])
    return avg_f1

## Determine Optimal Stacked Features

In [44]:
counter = 0
for max_feats in [True, False]:
    for min_feats in [True, False]:
        for average_feats in [True, False]:
            for binary_feats in [True,False]:
                for combo_feats in [True,False]:
                    if not any([max_feats, min_feats, average_feats, binary_feats, combo_feats]):
                        continue
                        
                    counter +=1
                    micro_f1 = train_stacked_classifier(dual=True, penalty='l2', C=1.0, \
                                                      max_feats=max_feats, min_feats=min_feats, average_feats=average_feats,\
                                                      binary_feats=binary_feats, combo_feats=combo_feats)
                    print("{ts} {counter} MICRO: F1: {f1:.6f} Max:{max} Min:{min} Avg:{average} Binary:{binary} Combo:{combo}".format(\
                        ts=get_ts(), counter=counter, f1=micro_f1, max=max_feats, min=min_feats, \
                        average=average_feats, binary=binary_feats, combo=combo_feats))

2019-10-09 04:56:42.389669 1 MICRO: F1: 0.766317 Max:True Min:True Avg:True Binary:True Combo:True
2019-10-09 04:58:41.246004 2 MICRO: F1: 0.761317 Max:True Min:True Avg:True Binary:True Combo:False
2019-10-09 05:00:39.949236 3 MICRO: F1: 0.766217 Max:True Min:True Avg:True Binary:False Combo:True
2019-10-09 05:02:33.723245 4 MICRO: F1: 0.756385 Max:True Min:True Avg:True Binary:False Combo:False
2019-10-09 05:04:32.705545 5 MICRO: F1: 0.766185 Max:True Min:True Avg:False Binary:True Combo:True
2019-10-09 05:06:30.012990 6 MICRO: F1: 0.760728 Max:True Min:True Avg:False Binary:True Combo:False
2019-10-09 05:08:26.350415 7 MICRO: F1: 0.765720 Max:True Min:True Avg:False Binary:False Combo:True
2019-10-09 05:10:18.556637 8 MICRO: F1: 0.756692 Max:True Min:True Avg:False Binary:False Combo:False
2019-10-09 05:12:15.718415 9 MICRO: F1: 0.766717 Max:True Min:False Avg:True Binary:True Combo:True
2019-10-09 05:14:11.749650 10 MICRO: F1: 0.762626 Max:True Min:False Avg:True Binary:True Combo:

In [45]:
#Best 2017-12-04 02:47:40.843997 11 MICRO: F1: 0.763158 Max:True Min:False Avg:True Binary:False Combo:True

## - GOT UP TO HERE?

In [46]:
optimal_feats = 65
max_feats = 85

(max_feats-optimal_feats)/max_feats, optimal_feats/max_feats

(0.23529411764705882, 0.7647058823529411)

### Best Features are Max Prob, Binary and Combo

In [47]:
counter = 0
for dual in [True, False]:
    for penalty in ["l1", "l2"]:
        # dual only support l2
        if dual and penalty != "l2":
            continue
        for C in [0.1, 0.5, 1.0, 5.0, 10.0, 100.0]:
            counter += 1
            micro_f1 = train_stacked_classifier(dual=dual, penalty=penalty, C=C, \
                                                      max_feats=True, min_feats=False, average_feats=True,\
                                                      binary_feats=False, combo_feats=True)                
            print("%i MICRO: F1: %s dual: %s penalty: %s C:%s"
                   % (counter, str(round(micro_f1, 6)).rjust(8), str(dual), str(penalty), str(round(C, 3)).rjust(5)))

1 MICRO: F1: 0.753255 dual: True penalty: l2 C:  0.1
2 MICRO: F1: 0.764977 dual: True penalty: l2 C:  0.5
3 MICRO: F1: 0.766026 dual: True penalty: l2 C:  1.0
4 MICRO: F1: 0.763994 dual: True penalty: l2 C:  5.0
5 MICRO: F1: 0.762851 dual: True penalty: l2 C: 10.0
6 MICRO: F1: 0.751405 dual: True penalty: l2 C:100.0
7 MICRO: F1: 0.752019 dual: False penalty: l1 C:  0.1
8 MICRO: F1: 0.763073 dual: False penalty: l1 C:  0.5
9 MICRO: F1: 0.761854 dual: False penalty: l1 C:  1.0
10 MICRO: F1: 0.760401 dual: False penalty: l1 C:  5.0
11 MICRO: F1: 0.757682 dual: False penalty: l1 C: 10.0
12 MICRO: F1: 0.743817 dual: False penalty: l1 C:100.0
13 MICRO: F1: 0.753255 dual: False penalty: l2 C:  0.1
14 MICRO: F1: 0.764977 dual: False penalty: l2 C:  0.5
15 MICRO: F1: 0.766026 dual: False penalty: l2 C:  1.0
16 MICRO: F1: 0.763994 dual: False penalty: l2 C:  5.0
17 MICRO: F1: 0.762556 dual: False penalty: l2 C: 10.0
18 MICRO: F1: 0.754259 dual: False penalty: l2 C:100.0


In [48]:
# best:  2017-12-4 - 3 MICRO: F1: 0.763158 dual: True penalty: l2 C:  1.0

## Compute Test Metric Performance

In [49]:
td = []
td_ys = defaultdict(list)
for i in range(CV_FOLDS):
    tmp_xs = predicts_by_fold[i]
    td.append(tmp_xs)

    _, _, _, td_ys_by_tag_cr_sent, _ = fold2test_data[i]
    merge_dictionaries(td_ys_by_tag_cr_sent, td_ys)

xs = np.vstack(td)
# ensure the same number of rows
xs.shape, len(list(td_ys.values())[0])

((8573, 119, 10), 8573)

In [50]:
xs_feats_train = get_stacked_feat_from_probs(xs,  max_feats=True, min_feats=False, average_feats=True,\
                                                      binary_feats=False, combo_feats=True)
xs_feats_train.shape

(8573, 65)

In [51]:
xs_feats_test = get_stacked_feat_from_probs(test_probs,  max_feats=True, min_feats=False, average_feats=True,\
                                                      binary_feats=False, combo_feats=True)
xs_feats_test.shape

(2097, 65)

In [52]:
np.mean(xs_feats_train), np.mean(xs_feats_test), np.max(xs_feats_train), np.max(xs_feats_test)

(0.079120603580585855, 0.075166455496904067, 1.0, 1.0)

In [53]:
#optimal from earlier
fn_create_sent_cls = lambda: LogisticRegression(dual=True, C=1.0, penalty='l2')
tag2sent_classifier = train_classifier_per_code(xs_feats_train, td_ys , fn_create_sent_cls, cr_vtags, verbose=False)
train_sent_predictions_by_code \
    = test_classifier_per_code(xs_feats_train, tag2sent_classifier, cr_vtags )

test_sent_predictions_by_code \
    = test_classifier_per_code(xs_feats_test, tag2sent_classifier, cr_vtags )

In [54]:
CB_SENT_TD, CB_SENT_VD = "TEST_CR_SC_STACKED_TD", "TEST_CR_SC_STACKED_VD"
parameters = dict(config)
parameters["extractors"] = []
# merge in function args
sent_algo = "stacked"
parameters.update({'dual': True, 'C':10.0, 'penalty':'l2', 
                   'max_feats': True, 'min_feats': False, 'average_feats': False, 
                   'binary_feats': True, 'combo_feats': True})

sent_td_objectid = processor.persist_results(CB_SENT_TD, td_ys,                  train_sent_predictions_by_code, parameters, sent_algo)
sent_vd_objectid = processor.persist_results(CB_SENT_VD, test_ys_by_tag_cr_sent, test_sent_predictions_by_code,  parameters, sent_algo)